In [1]:
N1=0xa4d80845630d3b332f74f667ec8a0e49aba15b6f0c4f4006161d62c91b78cf6811421cc76609d2d9dba2c43be9d8ecdc6a0dff64a8041dcde52c7f92820b0a38fc91419e8ec9a5c69d47edc6e347934b4d87f97c5759886dac6c1143ff55b8eb11acfaa6cc70956a8ec7796e1a063b123bc2e467e30937c5a69c7ab5f8ed17e1;
e1=0x3458c2e97adef45f741c7db11ece6c0814aa5b6fad9144242cdaa16a6b4f3622477935f98a41765b92892b4de22a391cf08767447df113f5151c86edd109b97f9b045fd8ad5d7a51084684d4e2353db6c0e474d5d79f399a2bf4fd867ec85b7960845ab5497f705914912f797804c06dcff57139e040596d22b141e54835e0d3;
c1=0x91b097a5b1f6b12accdbda15cd2247384e1b3ed8311085a0f3e0dbb5fffce650a355600a02674189d1b7f4075df079c70354a08646e85ecf31dd150220cd1d4ce22d55a946500f4bd8def74fb0acea3e8d2e7bb1d27ebf2ca2e80fc28c3f0d88a041d4a556a18147f66b88c65f19c99b4b94c3f78d468b8accb4da7e7ce31b29;
#私钥d1的取值范围为[2**249,2**250]

def rational_to_contfrac(x,y):
    # Converts a rational x/y fraction into 
    # a list of partial quotients [a0, ..., an]
    a = x // y
    pquotients = [a]
    while a * y != x:
        x, y = y, x - a * y
        a = x // y
        pquotients.append(a)
    return pquotients

def convergents_from_contfrac(frac):
    # computes the list of convergents 
    # using the list of partial quotients
    convs = [];
    for i in range(len(frac)): convs.append(contfrac_to_rational(frac[0 : i]))
    return convs

def contfrac_to_rational (frac):
    # Converts a finite continued fraction 
    # [a0, ..., an] to an x/y rational.
    if len(frac) == 0: return (0,1)
    num = frac[-1]
    denom = 1
    for _ in range(-2, -len(frac) - 1, -1): num, denom = frac[_] * num + denom, num
    return (num, denom)

def egcd(a, b):
    if a == 0: return (b, 0, 1)
    g, x, y = egcd(b % a, a)
    return (g, y - (b // a) * x, x)

def mod_inv(a, m):
    g, x, _ = egcd(a, m)
    return (x + m) % m

def isqrt(n):
    x = n
    y = (x + 1) // 2
    while y < x:
        x = y
        y = (x + n // x) // 2
    return x

def crack_rsa(e, n):
    frac = rational_to_contfrac(e, n)
    convergents = convergents_from_contfrac(frac)

    for (k, d) in convergents:
        if k != 0 and (e * d - 1) % k == 0:
            phi = (e * d - 1) // k
            s = n - phi + 1 #s = p+q
            # check if x*x - s*x + n = 0 has integer roots
            D = s * s - 4 * n
            if D >= 0:
                sq = isqrt(D) #sq  = p-q
                if sq * sq == D and (s + sq) % 2 == 0:
                    print('s =',s,'\nsq =',sq)
                    #s = 21598774039586200070147525474916319923783370208965506859966474034192466891752071825113031402816286211060205173453261716356306698126753739379314851803088562
                    #sq = 1864889837813344854107018271943910799305519977071181551049758492320620160876635384290248618975794787397350578373088872637762596794152758026944033957689784
                    return d

d = crack_rsa(e, n)
m = hex(pow(c, d, n))[2:]
s = 21598774039586200070147525474916319923783370208965506859966474034192466891752071825113031402816286211060205173453261716356306698126753739379314851803088562
sq = 1864889837813344854107018271943910799305519977071181551049758492320620160876635384290248618975794787397350578373088872637762596794152758026944033957689784
p = (s + sq) / 2
q = s - p
print("found q: %d" % q)
print("found p: %d" % p)
print("found d: %d" % d)
print("found m: %d" % int('0x'+m,16))
print(bytes.fromhex(m)[::-1])

"""
found q: 9866942100886426502240730997753957451396363051135856228400890124472226574139375372709352198992370806050549635710824201176393352751670502703990849207795712
found p: 11731831938699772462127271873430115361544445093018344205508116263256543526314353604701640010896040499228777875913175294497034647460453248703129442880323584
found d: 1118059191747025979407667430202387068489815082618058538672641608386136195643
found m: 72115607506154761336411014109985685056395095747321491967349423247749565387913611968183729328564132391301926185639936696286792673324141050046544603161058262375131485972136960821908946097232689498056450728225835250747872903494693577037
b'Mathematics is the queen of sciences, and arithmetic [number theory] is the queen of mathematics.'
"""

s = 21598774039586200070147525474916319923783370208965506859966474034192466891752071825113031402816286211060205173453261716356306698126753739379314851803088562 
sq = 1864889837813344854107018271943910799305519977071181551049758492320620160876635384290248618975794787397350578373088872637762596794152758026944033957689784
found q: 9866942100886427608020253601486204562238925115947162654458357770935923365437718220411391391920245711831427297540086421859272050666300490676185408922699389
found p: 11731831938699772462127271873430115361544445093018344205508116263256543526314353604701640010896040499228777875913175294497034647460453248703129442880389173
found d: 1118059191747025979407667430202387068489815082618058538672641608386136195643
found m: 72115607506154761336411014109985685056395095747321491967349423247749565387913611968183729328564132391301926185639936696286792673324141050046544603161058262375131485972136960821908946097232689498056450728225835250747872903494693577037
b'Mathematics is the 

"\nfound q: 9866942100886426502240730997753957451396363051135856228400890124472226574139375372709352198992370806050549635710824201176393352751670502703990849207795712\nfound p: 11731831938699772462127271873430115361544445093018344205508116263256543526314353604701640010896040499228777875913175294497034647460453248703129442880323584\nfound d: 1118059191747025979407667430202387068489815082618058538672641608386136195643\nfound m: 72115607506154761336411014109985685056395095747321491967349423247749565387913611968183729328564132391301926185639936696286792673324141050046544603161058262375131485972136960821908946097232689498056450728225835250747872903494693577037\nb'Mathematics is the queen of sciences, and arithmetic [number theory] is the queen of mathematics.'\n"

In [1]:
from time import *
from Crypto.Util.number import *
from gmpy2 import iroot

from sage.all import pari
from subprocess import check_output
from re import findall

# 增加 PARI 堆栈大小
pari.allocatemem(2**30 * 58)

def flatter(M):
    z = "[[" + "]\n[".join(" ".join(map(str, row)) for row in M) + "]]"
    ret = check_output(["flatter"], input=z.encode())
    flattened_values = list(map(int, findall(b"-?\\d+", ret)))
    return matrix(M.nrows(), M.ncols(), flattened_values)


debug = True
helpful_only = True
dimension_min = 7  # stop removing if lattice reaches that dimension


def helpful_vectors(BB, modulus):
    nothelpful = 0
    for ii in range(BB.dimensions()[0]):
        if BB[ii, ii] >= modulus:
            nothelpful += 1

    print(nothelpful, "/", BB.dimensions()[0], " vectors are not helpful")


# tries to remove unhelpful vectors
# we start at current = n-1 (last vector)
def remove_unhelpful(BB, monomials, bound, current):
    # end of our recursive function
    if current == -1 or BB.dimensions()[0] <= dimension_min:
        return BB

    # we start by checking from the end
    for ii in range(current, -1, -1):
        # if it is unhelpful:
        if BB[ii, ii] >= bound:
            affected_vectors = 0
            affected_vector_index = 0
            # let's check if it affects other vectors
            for jj in range(ii + 1, BB.dimensions()[0]):
                # if another vector is affected:
                # we increase the count
                if BB[jj, ii] != 0:
                    affected_vectors += 1
                    affected_vector_index = jj

            # level:0
            # if no other vectors end up affected
            # we remove it
            if affected_vectors == 0:
                print("* removing unhelpful vector", ii)
                BB = BB.delete_columns([ii])
                BB = BB.delete_rows([ii])
                monomials.pop(ii)
                BB = remove_unhelpful(BB, monomials, bound, ii - 1)
                return BB

            elif affected_vectors == 1:
                affected_deeper = True
                for kk in range(affected_vector_index + 1, BB.dimensions()[0]):
                    # if it is affecting even one vector
                    # we give up on this one
                    if BB[kk, affected_vector_index] != 0:
                        affected_deeper = False
                # remove both it if no other vector was affected and
                # this helpful vector is not helpful enough
                # compared to our unhelpful one
                if affected_deeper and abs(bound - BB[affected_vector_index, affected_vector_index]) < abs(
                        bound - BB[ii, ii]):
                    print("* removing unhelpful vectors", ii, "and", affected_vector_index)
                    BB = BB.delete_columns([affected_vector_index, ii])
                    BB = BB.delete_rows([affected_vector_index, ii])
                    monomials.pop(affected_vector_index)
                    monomials.pop(ii)
                    BB = remove_unhelpful(BB, monomials, bound, ii - 1)
                    return BB
    # nothing happened
    return BB


def boneh_durfee(pol, A, modulus, mm, tt, XX, YY):
    PR.< u, x, y > = PolynomialRing(ZZ)
    Q = PR.quotient(x * y + A - u)  # u = xy + A
    polZ = Q(pol).lift()
    print(polZ)
    UU = XX * YY + A

    # x-shifts
    gg = []
    for kk in range(mm + 1):
        for ii in range(mm - kk + 1):
            xshift = x ^ ii * modulus ^ (mm - kk) * polZ(u, x, y) ^ kk
            gg.append(xshift)
    gg.sort()

    # x-shifts list of monomials
    monomials = []
    for polynomial in gg:
        for monomial in polynomial.monomials():
            if monomial not in monomials:
                monomials.append(monomial)
    monomials.sort()

    # y-shifts (selected by Herrman and May)
    for jj in range(1, tt + 1):
        for kk in range(floor(mm / tt) * jj, mm + 1):
            yshift = y ^ jj * polZ(u, x, y) ^ kk * modulus ^ (mm - kk)
            yshift = Q(yshift).lift()
            gg.append(yshift)  # substitution

    # y-shifts list of monomials
    for jj in range(1, tt + 1):
        for kk in range(floor(mm / tt) * jj, mm + 1):
            monomials.append(u ^ kk * y ^ jj)

    # construct lattice B
    nn = len(monomials)
    BB = Matrix(ZZ, nn)
    for ii in range(nn):
        BB[ii, 0] = gg[ii](0, 0, 0)
        for jj in range(1, ii + 1):
            if monomials[jj] in gg[ii].monomials():
                BB[ii, jj] = gg[ii].monomial_coefficient(monomials[jj]) * monomials[jj](UU, XX, YY)

    # Prototype to reduce the lattice
    if helpful_only:
        BB = remove_unhelpful(BB, monomials, modulus ^ mm, nn - 1)
        # reset dimension
        nn = BB.dimensions()[0]
        if nn == 0:
            print("failure")
            return 0, 0

    # check if vectors are helpful
    if debug:
        helpful_vectors(BB, modulus ^ mm)

    # check if determinant is correctly bounded
    det = BB.det()
    bound = modulus ^ (mm * nn)
    if det >= bound:
        return 0, 0
    else:
        print("det(L) < e^(m*n) (good! If a solution exists < N^delta, it will be found)")

    start = time()
    BB = flatter(BB)
    end = time()
    print( "lll: %s s" % (end-start))
    if debug:
        print("looking for independent vectors in the lattice")
    found_polynomials = False

    for pol1_idx in range(nn - 1):
        for pol2_idx in range(pol1_idx + 1, nn):
            # for i and j, create the two polynomials
            PR.< w, z > = PolynomialRing(ZZ)
            pol1 = pol2 = 0
            for jj in range(nn):
                pol1 += monomials[jj](w * z + A, w, z) * BB[pol1_idx, jj] / monomials[jj](UU, XX, YY)
                pol2 += monomials[jj](w * z + A, w, z) * BB[pol2_idx, jj] / monomials[jj](UU, XX, YY)

            # resultant
            PR.< q > = PolynomialRing(ZZ)
            rr = pol1.resultant(pol2)

            # are these good polynomials?
            if rr.is_zero() or rr.monomials() == [1]:
                continue
            else:
                print("found them, using vectors", pol1_idx, "and", pol2_idx)
                found_polynomials = True
                break
        if found_polynomials:
            break

    if not found_polynomials:
        print(f'no independant vectors could be found. This should very rarely happen...')
        return 0, 0

    rr = rr(q, q)

    # solutions
    soly = rr.roots()

    if len(soly) == 0:
        print("Your prediction (delta) is too small")
        return 0, 0

    soly = soly[0][0]
    print(f'find y = {soly}')
    ss = pol1(q, soly)
    solx = ss.roots()[0][0]
    print(f'find x = {solx}')
    return solx, soly


def example():
    N = 0x94e4c83c67c6d6e33d83cc2953df899e8c4b33894f653d5bbc84d7dd9058e6949221897f6e5b7b8bd9013f495c906862e401436e77be585474066f6c220751dd9b2b8be66f07ad7f090547a6e759e482ba263b941b32c27c62c4b558d96dda168b28c52e550b7d7ff145a5996c0b398714cf5ee8f0ea1a3d5b17c592f1c15275
    n = 1024
    e = 0x949b2e72766be1e83ee278a56bc86a2d3268b719507068ac62c6d249a810284edaac39335e8d699630887c13864f4cdf1c0c423b2f7ae88ccc60a827332e6c410800c7c7a1677918c28aa51086991d1290fc64b8e1b0f14b482f35d86139bb3491a59e2ad99dcd35bd129a44c3b8e2667e405dc2d307a5bb5a1504d7ded3bda3
    c = 0x6fd6fae8ab4e95e622e5dad2921c6f12e911df08768abf2d10d212ad9a26e4c5ec71640d7a6b3488064fd424224bc2c762b956af95a3212de37a57d74c0299936f48ae3d8b8803e644e8d1306ab735c94fd815fe8c77982b32d51e9b6f3b3d4f3753810b61fb528c3e9eb774dabd93a3c5c9919ae3fb90e8e998ed3e7f949738
    beta = 560
    d2 = 0x6da211f0d34b
    r = 437
    A = e * d2 - 1
    W = e << r

    P.< x, y > = PolynomialRing(ZZ)
    pol = A + x * (y - N)
    X = 2 ** 561
    Y = 2 ** 514

    m = 10
    tt = float((n + 2 * r - 2 * beta) * m / (2 * n))
    print(tt)
    if tt - int(tt) > 0.5:
        t = int(tt) + 1
    else:
        t = int(tt)
    print('tt =', tt, ', t = ', t, ', m = ', m)


    start_time = time()
    solx, soly = boneh_durfee(pol, A, W, m, t, X, Y)
    # found a solution?
    if solx > 0:
        print("=== solution found ===")
        d = abs(int(pol(solx, soly) / W))
        print("fake private key found:", d)
    else:
        print("=== no solution was found ===")

    print(("=== %s seconds ===" % (time() - start_time)))
    y = int(soly + 1)
    p = ((y) + iroot((y)**2 - 4*N,2)[0]) // 2
    q = y - p
    d = inverse(e,(p-1)*(q-1))
    m = pow(c,d,N)
    text = long_to_bytes(m)[::-1]
    print("found q: %d" % q)
    print("found p: %d" % p)
    print("found d: %d" % d)
    print("found m: %d" %m)
    print(text)


if __name__ == "__main__":
    example()



"""
lll: 6.783688545227051 s
looking for independent vectors in the lattice
found them, using vectors 0 and 1
find y = 20669116529972280868235472642422776201909637285884516155862888589837342513282316090088342096364702368094391219552254156670270390106105101130869387179628885
find x = 2444120606414954348576931306989550643749718095148281935395568529714227520357873483806743512053166862668913633899477775724325015913046587512804612884924361047516649344078
=== solution found ===
fake private key found: 6900076873138233730017493813729936353
=== 96.95609545707703 seconds ===
found q: 8835759918378718527006376140829267440252695435801133066894349216192062561096262594830004752243829417419301825565664373308334332806934765880895262721280249
found p: 11833356611593562341229096501593508761656941850083383088968539373645279952186053495258337344120872950675089393986589783361936057299170335249974124458348637
found d: 2448849156256549779765317391086514527271067466228669911975874547167020323323129472631075062718316334662053762900428433851806955386502040052243058227305021484420866036555
found m: 411708110548976412778536399221192829597879130418779704340440446197846122866994974250241237132115958563099708213990983544262962354754778176004782101155215298442496870041733756567944507725673757021127337172975026812709705824528341829542021957086677416458885678605362243282371896905
b'If people do not believe that mathematics is simple, it is only because they do not realize how complicated life is.'
"""

Be yourself.


In [ ]:
from time import *
from Crypto.Util.number import *
from gmpy2 import iroot

from sage.all import pari
from subprocess import check_output
from re import findall

# 增加 PARI 堆栈大小
pari.allocatemem(2**30 * 58)
def flatter(M):
    z = "[[" + "]\n[".join(" ".join(map(str, row)) for row in M) + "]]"
    ret = check_output(["flatter"], input=z.encode())
    flattened_values = list(map(int, findall(b"-?\\d+", ret)))
    return matrix(M.nrows(), M.ncols(), flattened_values)
debug = True
strict = False
helpful_only = True
dimension_min = 15
def helpful_vectors(BB, modulus):
    nothelpful = 0
    for ii in range(BB.dimensions()[0]):
        if BB[ii,ii] >= modulus:
            nothelpful += 1
def matrix_overview(BB, bound):
    for ii in range(BB.dimensions()[0]):
        a = ('%02d ' % ii)
        for jj in range(BB.dimensions()[1]):
            a += '0' if BB[ii,jj] == 0 else 'X'
            if BB.dimensions()[0] < 60:
                a += ' '
        if BB[ii, ii] >= bound:
            a += '~'
def remove_unhelpful(BB, monomials, bound, current):
    if current == -1 or BB.dimensions()[0] <= dimension_min:
        return BB
    for ii in range(current, -1, -1):
        if BB[ii, ii] >= bound:
            affected_vectors = 0
            affected_vector_index = 0
            for jj in range(ii + 1, BB.dimensions()[0]):
                if BB[jj, ii] != 0:
                    affected_vectors += 1
                    affected_vector_index = jj
            if affected_vectors == 0:
                BB = BB.delete_columns([ii])
                BB = BB.delete_rows([ii])
                monomials.pop(ii)
                BB = remove_unhelpful(BB, monomials, bound, ii-1)
                return BB
            elif affected_vectors == 1:
                affected_deeper = True
                for kk in range(affected_vector_index + 1, BB.dimensions()[0]):
                    if BB[kk, affected_vector_index] != 0:
                        affected_deeper = False
                if affected_deeper and abs(bound - BB[affected_vector_index, affected_vector_index]) < abs(bound - BB[ii, ii]):
                    BB = BB.delete_columns([affected_vector_index, ii])
                    BB = BB.delete_rows([affected_vector_index, ii])
                    monomials.pop(affected_vector_index)
                    monomials.pop(ii)
                    BB = remove_unhelpful(BB, monomials, bound, ii-1)
                    return BB
    return BB
def boneh_durfee(pol, modulus, mm, tt, XX, YY):
    PR.<u, x, y> = PolynomialRing(ZZ)
    Q = PR.quotient(x*y + 1 - u)
    polZ = Q(pol).lift()
    UU = XX*YY + 1
    gg = []
    for kk in range(mm + 1):
        for ii in range(mm - kk + 1):
            xshift = x^ii * modulus^(mm - kk) * polZ(u, x, y)^kk
            gg.append(xshift)
    gg.sort()
    monomials = []
    for polynomial in gg:
        for monomial in polynomial.monomials():
            if monomial not in monomials:
                monomials.append(monomial)
    monomials.sort()
    for jj in range(1, tt + 1):
        for kk in range(floor(mm/tt) * jj, mm + 1):
            yshift = y^jj * polZ(u, x, y)^kk * modulus^(mm - kk)
            yshift = Q(yshift).lift()
            gg.append(yshift)
    for jj in range(1, tt + 1):
        for kk in range(floor(mm/tt) * jj, mm + 1):
            monomials.append(u^kk * y^jj)
    nn = len(monomials)
    BB = Matrix(ZZ, nn)
    for ii in range(nn):
        BB[ii, 0] = gg[ii](0, 0, 0)
        for jj in range(1, ii + 1):
            if monomials[jj] in gg[ii].monomials():
                BB[ii, jj] = gg[ii].monomial_coefficient(monomials[jj]) * monomials[jj](UU,XX,YY)
    if helpful_only:
        BB = remove_unhelpful(BB, monomials, modulus^mm, nn-1)
        nn = BB.dimensions()[0]
        if nn == 0:
            return 0,0
    det = BB.det()
    bound = modulus^(mm*nn)
    if det >= bound:
        if debug:
            diff = (log(det) - log(bound)) / log(2)
        if strict:
            return -1, -1
    else:
        print("det(L) < e^(m*n) (good! If a solution exists < N^delta, it will be found)")
    BB = flatter(BB)
    found_polynomials = False
    for pol1_idx in range(nn - 1):
        for pol2_idx in range(pol1_idx + 1, nn):
            PR.<w,z> = PolynomialRing(ZZ)
            pol1 = pol2 = 0
            for jj in range(nn):
                pol1 += monomials[jj](w*z+1,w,z) * BB[pol1_idx, jj] / monomials[jj](UU,XX,YY)
                pol2 += monomials[jj](w*z+1,w,z) * BB[pol2_idx, jj] / monomials[jj](UU,XX,YY)
            PR.<q> = PolynomialRing(ZZ)
            rr = pol1.resultant(pol2)
            if rr.is_zero() or rr.monomials() == [1]:
                continue
            else:
                found_polynomials = True
                break
        if found_polynomials:
            break
    if not found_polynomials:
        return 0, 0
    rr = rr(q, q)
    soly = rr.roots()
    if len(soly) == 0:
        return 0, 0
    soly = soly[0][0]
    ss = pol1(q, soly)
    solx = ss.roots()[0][0]
    return solx, soly

def example():
    ############################################
    # How To Use This Script
    ##########################################

    #
    # The problem to solve (edit the following values)
    #

    # the modulus
    N = 0xd231f2c194d3971821984dec9cf1ef58d538975f189045ef8a706f6165aab4929096f61a3eb7dd8021bf3fdc41fe3b3b0e4ecc579b4b5e7e035ffcc383436c9656533949881dca67c26d0e770e4bf62a09718dbabc2b40f2938f16327e347f187485aa48b044432e82f5371c08f6e0bbde46c713859aec715e2a2ca66574f3eb
    # the public exponent
    e = 0x5b5961921a49e3089262761e89629ab6dff2da1504a0e5eba1bb7b20d63c785a013fd6d9e021c01baf1b23830954d488041b92bca2fe2c92e3373dedd7e625da11275f6f18ee4aef336d0637505545f70f805902ddbacb21bb8276d34a0f6dfe37ede87dd95bb1494dbb5763639ba3984240f1178e32aa36ee3c5fcc8115dde5
    c = 0x6a88a8fa2b8f28d96284298bab2061efeb35e3a086370e19523c15c429f5d783b9d4f32e31a402916f45ad4f2760ab30e77177335af44756bfbeef0f168b5e0dc8c3ddf75d141c358969cca0e7c2b8ab99ef8e33b031be1cbccd95b687682ac7b0dcc0d56f5651ee671d6358128d2e0801f247a6af4fe0dc5e8fb199eba0780f
    # the hypothesis on the private exponent (the theoretical maximum is 0.292)
    delta = 0.279
    #
    # Lattice (tweak those values)
    #

    # you shld tweak this (after a first run), (e.g. increment it until a solution is found)
    m = 11

    # you need to be a lattice master to tweak these
    t = int((1-2*delta) * m)  # optimization from Herrmann and May
    X = 2*floor(N^delta)  # this _might_ be too much
    Y = floor(N^(1/2))    # correct if p, q are ~ same size

    #
    # Don't touch anything below
    #

    # Problem put in equation
    P.<x,y> = PolynomialRing(ZZ)
    A = int((N+1)/2)
    pol = 1 + x * (A + y)

    #
    # Find the solutions!
    #

    # Checking bounds
    if debug:
        print("=== checking values ===")
        print("* delta:", delta)
        print("* delta < 0.292", delta < 0.292)
        print("* size of e:", int(log(e)/log(2)))
        print("* size of N:", int(log(N)/log(2)))
        print("* m:", m, ", t:", t)

    # boneh_durfee
    if debug:
        print("=== running algorithm ===")
        start_time = time()

    solx, soly = boneh_durfee(pol, e, m, t, X, Y)

    # found a solution?
    if solx > 0:
        print("=== solution found ===")
        if False:
            print("x:", solx)
            print("y:", soly)

        d = int(pol(solx, soly) / e)
        print("private key found:", d)
        print("soly=",soly)

    else:
        print("=== no solution was found ===")

    if debug:
        print(("=== %s seconds ===" % (time() - start_time)))
        y = int(-soly * 2)
        print('p+q=',y)
        p = ((y) + iroot((y)**2 - 4*N,2)[0]) // 2
        q = y - p
        m = pow(c,d,N)
        text = long_to_bytes(m)[::-1]
        print("found q: %d" % q)
        print("found p: %d" % p)
        print("found d: %d" % d)
        print("found m: %d" %m)
        print(text)
if __name__ == "__main__":
    example()


"""
lll: 5.586979150772095 s
LLL is done!
looking for independent vectors in the lattice
found them, using vectors 0 and 1
=== solution found ===
private key found: 96884485470809436429011642866838703862114673941017100586009098363273491491514356269093
soly= -12152534193309507966154462609807876697411353511577001844941780164885228159689077784979280989619680779398111826859207192887084264330276793427668205533319626
=== 96.20228338241577 seconds ===
p+q= 24305068386619015932308925219615753394822707023154003689883560329770456319378155569958561979239361558796223653718414385774168528660553586855336411066639252
found q: 11869583389521634999588895814931899209282833085315990491955438271004312624703796644214104875335462616923607689499528893181399330078966918201596985360139071
found p: 12435484997097380932720029404683854185539873937838013197928122058766143694674358925744457103903898941872615964218885492592769198581586668653739425706500181
found d: 96884485470809436429011642866838703862114673941017100586009098363273491491514356269093
found m: 467953840882317181499103662991214298422064486570574390714467505707261187314252509296933072417732732809103029658160034371
b'Cryptography is typically bypassed,not penetrated.'
"""

In [3]:
from gmpy2 import *
from Crypto.Util.number import *


# 1, get d
p = 10297529403524403127640670200603184608844065065952536889
a = 2
G = (8879931045098533901543131944615620692971716807984752065, 4106024239449946134453673742202491320614591684229547464)
d = (a*G[0]^3+G[1]^3+1)*inverse(G[1]*G[0], p)%p
Q = (6784278627340957151283066249316785477882888190582875173, 6078603759966354224428976716568980670702790051879661797)

# 2, get curve
def get_curve(a, d):
    d_over_3 = d*inverse(3, p)%p
    denominator = a - d_over_3**3
    a0 = 1
    a1 = (-3 * d_over_3 * inverse(denominator, p))%p
    a3 = (-9 * inverse(denominator**2, p)) % p
    a2 = (-9 * d_over_3**2 * inverse(denominator**2, p))%p
    a4 = (-27 * d_over_3 * inverse(denominator**3, p))%p
    a6 = (-27 * inverse(denominator**4, p))%p
    return [a1, a2, a3, a4, a6]
aa = get_curve(a, d)
# print(aa)
E = EllipticCurve(GF(p), aa)
# print(E)

# 3, TH to Wei
def toWei(M):
    '''
    https://www.hyperelliptic.org/EFD/g1p/data/twistedhessian/coordinates
    toweierstrass u = (-3 / (a - d d d/27)) x / (d x/3 - (-y) + 1)
    toweierstrass v = (-9 / ((a - d d d/27) (a - d d d/27))) (-y) / (d x/3 - (-y) + 1)
    '''
    x, y = M
    L = inverse(d*x*inverse(3,p) - (-y) + 1, p)
    M = (a - d^3*inverse(27, p))
    u = ((-3 * inverse(M, p)*x * L)) % p
    v = ((-9 * inverse(M^2, p) ) * (-y) * L)%p
    return (u, v)


G = E(toWei(G))
Q = E(toWei(Q))

# 4, get flag
long_to_bytes(int(Q.log(G)))


b'e@sy_cuRvL_c0o!'

In [2]:
from Crypto.Util.number import *
from tqdm import * 
c = 356435791209686635044593929546092486613929446770721636839137
p = 898278915648707936019913202333
q = 814090608763917394723955024893
n = p * q
m = b'X\xee\x1ey\x88\x01dX\xf6i\x91\x80h\xf4\x1f!\xa7"\x0c\x9a\x06\xc8\x06\x81\x15'
m = bytes_to_long(m)
begin = bytes_to_long(b"DASCTF{")
end = bytes_to_long(b"}")
for i in range(2000):
    F1 = begin * 256 ^ (i + 1)
    F2 = end
    K = F1 + F2 - m
    T = 256 ^ i
    M = matrix(ZZ, [[1, 0, 256 * T], [0, T, K * T], [0, 0, n * T]])
    L = M.LLL()
    for l in L:
        if l[-1] == 0:
            flag = int(abs(l[0]))
            if all(0<=j<=128 for j in long_to_bytes(flag)):
                print(long_to_bytes(flag))
                exit()

In [1]:
from Crypto.Util.number import *
from tqdm import *
from gmpy2 import *


n = 18770575776346636857117989716700159556553308603827318013591587255198383129370907809760732011993542700529211200756354110539398800399971400004000898098091275284235225898698802555566416862975758535452624647017057286675078425814784682675012671384340267087604803050995107534481069279281213277371234272710195280647747033302773076094600917583038429969629948198841325080329081838681126456119415461246986745162687569680825296434756908111148165787768172000131704615314046005916223370429567142992192702888820837032850104701948658736010527261246199512595520995042205818856177310544178940343722756848658912946025299687434514029951
c = 2587907790257921446754254335909686808394701314827194535473852919883847207482301560195700622542784316421967768148156146355099210400053281966782598551680260513547233270646414440776109941248869185612357797869860293880114609649325409637239631730174236109860697072051436591823617268725493768867776466173052640366393488873505207198770497373345116165334779381031712832136682178364090547875479645094274237460342318587832274304777193468833278816459344132231018703578274192000016560653148923056635076144189403004763127515475672112627790796376564776321840115465990308933303392198690356639928538984862967102082126458529748355566
p = next_prime(iroot(n, 2)[0])
q = n // p
# print(prime_pi(5))
e = prime_pi(703440151)-2
n = p * q
phi = (p - 1) * (q - 1)
d = inverse_mod(e, phi)
print(long_to_bytes(powmod(c, d, n)))

b'DASCTF{Ot2N63D_n8L6kJt_f40V61m_zS1O8L7}'
